In [ ]:
"""
Produce LandScape File
"""

con_db = {
    'HOST' : 'localhost', 'PORT' : '5433', 'USER' : 'postgres',
    'PASSWORD' : 'admin', 'DATABASE' : 'fireloc_db'
}

db_schema = {
    "GRID" : 'grid_ref', "GRID_PK" : 'gid', 'CELLID' : 'cellid',
    "DATA" : 'datasets', 'DATA_PK' : 'did', 'SLUG' : 'slug',
    "REL"  : 'rel_grid_data', 'GRID_FK' : 'gid', "DATA_FK" : 'did',
    "SHP"  : 'shp', 'RST' : 'rst'
}

main_folder = '/home/jasp/datasets'

epsg = 3763

lmt = '/home/jasp/tst_farsite/lmt.shp'

cos_slug = 'cos15_lvl3'
dem_slug = 'dem_m888'
tcd_slug = 'tcd_15'

procedure = '/home/jasp/tst_farsite/'

import os
from gasp.gt.prop.ext import get_ext
from gasp.g.to import shpext_to_boundary
from gasp.sql.fm import Q_to_df
from gasp.gt.nop.mos import rsts_to_mosaic
from gasp.gt.nop.rcls import rcls_rst
from gasp.gt.wenv.grs import run_grass

In [ ]:
lmt_geom = shpext_to_boundary(lmt).ExportToWkt()

Q = (
    "SELECT foo.{gid}, foo.{cellid}, jtbl.{slug_col}, "
    "jtbl.{rst}, jtbl.{shp} "
    "FROM ("
        "SELECT *, "
        "(ST_Area(ST_Intersection(geom, ST_GeomFromText('{wkt}', {srs}))) / ST_Area(geom) * 100) AS i_area "
        "FROM {gridt} "
        "WHERE ST_Intersects(geom, ST_GeomFromText('{wkt}', {srs}))"
    ") AS foo INNER JOIN ("
        "SELECT mtbl.{did}, mtbl.{slug_col}, "
        "jtbl.{relgid}, jtbl.{rst}, jtbl.{shp} "
        "FROM {datat} AS mtbl "
        "INNER JOIN {reltbl} AS jtbl ON "
        "mtbl.{did} = jtbl.{reldid} "
        "WHERE {slug_col}='{slug}' OR {slug_col}='{slug_dem}' "
        "OR {slug_col}='{slug_tcd}'"
    ") AS jtbl ON foo.{gid} = jtbl.{relgid} "
    "WHERE i_area > 1"
).format(
    gid=db_schema['GRID_PK'], cellid=db_schema['CELLID'],
    rst=db_schema['RST'], shp=db_schema['SHP'],
    wkt=lmt_geom, srs=str(epsg), gridt=db_schema["GRID"],
    datat=db_schema["DATA"],
    relgid=db_schema["GRID_FK"], reltbl=db_schema["REL"],
    did=db_schema["DATA_PK"], reldid=db_schema["DATA_FK"],
    slug=cos_slug, slug_col=db_schema["SLUG"],
    slug_dem=dem_slug, slug_tcd=tcd_slug
)

cells_df = Q_to_df(con_db, Q)
cells_df['rst'] = main_folder + '/' + cells_df.rst

In [ ]:
# Get COS_rst
cos_rst = cells_df[cells_df[db_schema["SLUG"]] == cos_slug].rst.tolist()

lulc_rst = rsts_to_mosaic(cos_rst, os.path.join(procedure, 'mos_lulc.tif'), api='rasterio')

# Reclass COS

# CLS to Fuel Model
rules = {
    (0, 11) : 90,
    (11, 19) : 1,
    20 : 5,
    21 : 1,
    22 : 9,
    23 : 6,
    24 : 1,
    25 : 6,
    26 : 8,
    27 : 90,
    (27, 32) : 98
}

rcls_rst = rcls_rst(lulc_rst, rules, os.path.join(procedure, 'rcls_lulc.tif'))

In [ ]:
# Get Tree Canocopy cover

tcd_rsts = cells_df[cells_df[db_schema["SLUG"]] == tcd_slug].rst.tolist()

tcd_rst = rsts_to_mosaic(tcd_rsts, os.path.join(procedure, 'tcd.tif'), api='rasterio')

# Reclass
rules = {0 : 99, (0, 20) : 1, (20, 50) : 2, (50, 80) : 3, (80, 100) : 4}

tcc_rst = rcls_rst(tcd_rst, rules, os.path.join(procedure, 'tcc.tif'))

In [ ]:
# Get DEM
dem_rsts = cells_df[cells_df[db_schema["SLUG"]] == dem_slug].rst.tolist()

dem_rst = rsts_to_mosaic(dem_rsts, os.path.join(procedure, 'dem_lulc.tif'), api='rasterio')

# Create Slope and Aspect Raster
grsbase = run_grass(procedure, grassBIN='grass78', location='dem_asp', srs=epsg)

import grass.script as grass
import grass.script.setup as gsetup
gsetup.init(grsbase, procedure, 'dem_asp', 'PERMANENT')

from gasp.gt.to.rst import rst_to_grs, grs_to_rst
from gasp.gt.wenv.grs import rst_to_region
from gasp.gt.nop.surf import slope, aspect

# Configure region
rst_ext = rst_to_grs(dem_rst, 'dem')
rst_to_region(rst_ext)

# Produce slope raster
dclv = slope(rst_ext, 'slope_rst', api='grass')

# Produce aspect raster
expo = aspect(rst_ext, 'aspect_rst', api='grass')

# Export data
dclv = grs_to_rst(dclv, os.path.join(procedure, 'dclv.tif'))
expo = grs_to_rst(expo, os.path.join(procedure, 'aspect.tif'))